In [6]:
# Purpose: Configure GENICAM_GENTL64_PATH so IDS Peak finds CTI
import os
import glob

# Optional: set explicit CTI directory here if known (edit this)
PEAK_CTI_HINT = None  # e.g., r"C:\\Program Files\\IDS\\ids_peak\\generic_sdk\\api\\bin\\x86_64"

def _ensure_gentl_path():  # ensure CTI path is set once
    existing = os.environ.get('GENICAM_GENTL64_PATH')  # get current env
    if existing:
        paths = [p for p in existing.split(';') if p]  # split semicolon list
        if any(os.path.isdir(p) and glob.glob(os.path.join(p, '*.cti')) for p in paths):
            return  # already valid
    found = []  # collect valid dirs with .cti
    if PEAK_CTI_HINT and os.path.isdir(PEAK_CTI_HINT) and glob.glob(os.path.join(PEAK_CTI_HINT, '*.cti')):
        found.append(PEAK_CTI_HINT)
    candidates = [  # common IDS Peak CTI locations (64-bit)
        r"C:\\Program Files\\IDS\\ids_peak",
        r"C:\\Program Files\\IDS",
    ]
    for root in candidates:
        if os.path.isdir(root):
            for d in glob.glob(os.path.join(root, '**'), recursive=True):
                if os.path.isdir(d) and glob.glob(os.path.join(d, '*.cti')):
                    found.append(d)
    if found:
        # de-duplicate while preserving order
        seen = set()
        uniq = []
        for d in found:
            if d not in seen:
                seen.add(d)
                uniq.append(d)
        os.environ['GENICAM_GENTL64_PATH'] = ';'.join(uniq)  # set env for this process
        print(f"GENICAM_GENTL64_PATH set to: {os.environ['GENICAM_GENTL64_PATH']}")  # single concise print
        return
    raise RuntimeError("GENICAM_GENTL64_PATH not found. Set to CTI directory (e.g., IDS Peak x86_64).")  # clear error

_ensure_gentl_path()  # run once

In [7]:
import cv2
import numpy as np
from ids_peak import ids_peak
from ids_peak_ipl import ids_peak_ipl
import json
import os
from collections import deque

# Global state for circle marking and controls
circle_state = {
    "center": None, 
    "radius": 30, 
    "dragging": False,
    "pixel_per_meter": None,  # calculated from camera specs
    "show_measurements": False,
    "ref_center": None,  # reference circle center
    "ref_radius": 100,  # reference circle radius (larger than main)
    "ref_dragging": False
}

# Camera control values
camera_controls = {
    "brightness": 100,
    "exposure": 5000,
    "analog_gain": 0,
    "digital_gain": 10  # stored as int (actual = value/10)
}

# Status message queue for GUI display
status_messages = deque(maxlen=5)  # keep last 5 messages

def add_status_message(msg):  # add message to status queue
    status_messages.append(msg)
    print(msg)  # also print to console

def mouse_callback(event, x, y, flags, param):  # handle mouse interactions for both circles
    if event == cv2.EVENT_LBUTTONDOWN:
        if circle_state["center"] is None:
            circle_state["center"] = (x, y)  # create main circle
            add_status_message(f"Main circle at: {circle_state['center']}")
        else:
            # Check if clicking on main circle
            dist_main = np.sqrt((x - circle_state["center"][0])**2 + (y - circle_state["center"][1])**2)
            if dist_main <= circle_state["radius"] + 10:
                circle_state["dragging"] = True  # drag main circle
            # Check if clicking on reference circle
            elif circle_state["ref_center"] is not None:
                dist_ref = np.sqrt((x - circle_state["ref_center"][0])**2 + (y - circle_state["ref_center"][1])**2)
                if dist_ref <= circle_state["ref_radius"] + 10:
                    circle_state["ref_dragging"] = True  # drag reference circle
            # Otherwise create/move reference circle
            else:
                circle_state["ref_center"] = (x, y)  # create reference circle
                add_status_message(f"Reference circle at: {circle_state['ref_center']}")
    elif event == cv2.EVENT_MOUSEMOVE:
        if circle_state["dragging"]:
            circle_state["center"] = (x, y)  # drag main circle
        elif circle_state["ref_dragging"]:
            circle_state["ref_center"] = (x, y)  # drag reference circle
    elif event == cv2.EVENT_LBUTTONUP:
        if circle_state["dragging"]:
            circle_state["dragging"] = False
            add_status_message(f"Main circle moved to: {circle_state['center']}")
        elif circle_state["ref_dragging"]:
            circle_state["ref_dragging"] = False
            add_status_message(f"Reference circle moved to: {circle_state['ref_center']}")
    elif event == cv2.EVENT_RBUTTONDOWN:
        # Right click clears both circles
        circle_state["center"] = None
        circle_state["ref_center"] = None
        circle_state["dragging"] = False
        circle_state["ref_dragging"] = False
        add_status_message("Circles cleared.")

def save_calibration(filename="calibration_mark.json"):  # save both circles position and settings
    if circle_state["center"] or circle_state["ref_center"]:
        data = {
            "center": circle_state["center"], 
            "radius": circle_state["radius"],
            "pixel_per_meter": circle_state["pixel_per_meter"],
            "ref_center": circle_state["ref_center"],
            "ref_radius": circle_state["ref_radius"]
        }
        with open(filename, 'w') as f:
            json.dump(data, f)
        add_status_message(f"Calibration saved to {filename}")

def load_calibration(filename="calibration_mark.json"):  # load both circles position and settings
    if os.path.exists(filename):
        with open(filename, 'r') as f:
            data = json.load(f)
        circle_state["center"] = tuple(data["center"]) if data.get("center") else None
        circle_state["radius"] = data.get("radius", 30)
        circle_state["pixel_per_meter"] = data.get("pixel_per_meter")
        circle_state["ref_center"] = tuple(data["ref_center"]) if data.get("ref_center") else None
        circle_state["ref_radius"] = data.get("ref_radius", 100)
        add_status_message(f"Calibration loaded from {filename}")

def nothing(x):  # dummy callback for trackbars
    pass

def calculate_pixel_per_meter(sensor_width_mm, image_width_px, focal_length_mm, distance_m):
    # sensor_width_mm: camera sensor width in mm
    # image_width_px: actual camera resolution width in pixels
    # focal_length_mm: lens focal length in mm
    # distance_m: distance from camera to object in meters
    # returns: pixels per meter at the given distance (width-based)
    sensor_width_m = sensor_width_mm / 1000.0  # convert to meters
    object_width_m = (sensor_width_m * distance_m) / focal_length_mm  # projected object width
    return image_width_px / object_width_m  # pixels per meter

def run_live_camera():
    # אתחול
    ids_peak.Library.Initialize()
    device_manager = ids_peak.DeviceManager.Instance()
   
    device = None
    data_stream = None
   
    try:
        device_manager.Update()
        if device_manager.Devices().empty():
            add_status_message("לא נמצאה מצלמה.")
            return

        # פתיחת מצלמה
        device = device_manager.Devices()[0].OpenDevice(ids_peak.DeviceAccessType_Control)
        nodemap = device.RemoteDevice().NodeMaps()[0]
       
        # פתיחת סטריים
        data_stream = device.DataStreams()[0].OpenDataStream()
       
        # הקצאת באפרים
        payload_size = nodemap.FindNode("PayloadSize").Value()
        for i in range(data_stream.NumBuffersAnnouncedMinRequired()):
            buffer = data_stream.AllocAndAnnounceBuffer(payload_size)
            data_stream.QueueBuffer(buffer)

        # התחלת צילום
        data_stream.StartAcquisition(ids_peak.AcquisitionStartMode_Default)
        nodemap.FindNode("AcquisitionStart").Execute()

        # Create GUI window and trackbars
        window_name = "IDS Peak - Calibration & Control"
        cv2.namedWindow(window_name)
        cv2.setMouseCallback(window_name, mouse_callback)
        
        # Create separate control panel windows for better visibility
        camera_window = "Camera Controls"
        calib_window = "Calibration Settings"
        cv2.namedWindow(camera_window)
        cv2.namedWindow(calib_window)
        
        # Resize windows to show full trackbar names
        cv2.resizeWindow(camera_window, 500, 250)  # wider window for camera controls
        cv2.resizeWindow(calib_window, 500, 350)  # wider window for calibration settings
        
        # Position windows adjacent to main camera view (attach to camera screen)
        cv2.moveWindow(window_name, 0, 0)  # main camera at top-left
        cv2.moveWindow(camera_window, 1300, 0)  # camera controls to the right
        cv2.moveWindow(calib_window, 1300, 250)  # calibration below camera controls
        
        # Camera Controls Window
        cv2.createTrackbar("Brightness", camera_window, 100, 2000, nothing)
        cv2.createTrackbar("Exposure", camera_window, 5000, 50000, nothing)
        cv2.createTrackbar("Analog Gain", camera_window, 0, 24, nothing)
        cv2.createTrackbar("Digital Gain x10", camera_window, 10, 80, nothing)
        
        # Calibration Settings Window
        cv2.createTrackbar("Radius", calib_window, 30, 200, nothing)
        cv2.createTrackbar("Ref Radius", calib_window, 100, 500, nothing)  # larger scale for reference
        cv2.createTrackbar("Measure", calib_window, 0, 1, nothing)
        cv2.createTrackbar("Notes", calib_window, 1, 1, nothing)
        cv2.createTrackbar("Sensor W mm", calib_window, 0, 50, nothing)
        cv2.createTrackbar("Focal mm", calib_window, 0, 200, nothing)
        cv2.createTrackbar("Dist cm", calib_window, 100, 500, nothing)
        cv2.createTrackbar("Width px", calib_window, 1280, 5000, nothing)

        add_status_message("המצלמה עובדת!")
        add_status_message("GUI: Camera Controls & Calibration Settings")
        add_status_message("Left-click: mark/drag circle | Right-click: clear")
        add_status_message("Keyboard: S-save | L-load | Q-quit")
        
        load_calibration()  # load previously saved calibration if exists
        
        # Set initial trackbar positions from loaded or default values
        cv2.setTrackbarPos("Radius", calib_window, circle_state["radius"])
        cv2.setTrackbarPos("Ref Radius", calib_window, circle_state["ref_radius"])

        while True:
            # קבלת באפר
            buffer = data_stream.WaitForFinishedBuffer(5000)
           
            # יוצרים תמונת IPL מהבאפר
            raw_image = ids_peak_ipl.Image.CreateFromSizeAndBuffer(
                buffer.PixelFormat(),
                buffer.BasePtr(),
                buffer.Size(),
                buffer.Width(),
                buffer.Height()
            )
           
            # המרה ל-BGR (צבע רגיל)
            converted_image = raw_image.ConvertTo(ids_peak_ipl.PixelFormatName_BGR8)
           
            # שליפת המערך ישירות כ-3D (גובה, רוחב, צבע)
            img_array = converted_image.get_numpy_3D()

            # הצגת התמונה
            img_array = cv2.resize(img_array, (1280, 960))
            img_height, img_width = img_array.shape[:2]
            
            # Read trackbar values
            brightness = cv2.getTrackbarPos("Brightness", camera_window)
            exposure = cv2.getTrackbarPos("Exposure", camera_window)
            analog_gain = cv2.getTrackbarPos("Analog Gain", camera_window)
            digital_gain_x10 = cv2.getTrackbarPos("Digital Gain x10", camera_window)
            circle_state["radius"] = cv2.getTrackbarPos("Radius", calib_window)
            circle_state["ref_radius"] = cv2.getTrackbarPos("Ref Radius", calib_window)
            show_measure = cv2.getTrackbarPos("Measure", calib_window)
            show_notes = cv2.getTrackbarPos("Notes", calib_window)
            
            # Camera spec trackbars for meter conversion
            sensor_w = cv2.getTrackbarPos("Sensor W mm", calib_window)
            focal_len = cv2.getTrackbarPos("Focal mm", calib_window)
            distance_cm = cv2.getTrackbarPos("Dist cm", calib_window)
            
            # Actual camera resolution (user can set this for accurate meter calculation)
            cam_width_px = cv2.getTrackbarPos("Width px", calib_window)
            
            # Update pixel per meter if specs are provided
            if sensor_w > 0 and focal_len > 0 and distance_cm > 0 and cam_width_px > 0:
                distance_m = distance_cm / 100.0
                circle_state["pixel_per_meter"] = calculate_pixel_per_meter(
                    sensor_w, cam_width_px, focal_len, distance_m
                )
            else:
                circle_state["pixel_per_meter"] = None  # reset if incomplete specs
            
            # Apply camera controls if changed
            if brightness != camera_controls["brightness"]:
                try:
                    nodemap.FindNode("Brightness").SetValue(brightness)
                    camera_controls["brightness"] = brightness
                except: pass
            
            if exposure != camera_controls["exposure"]:
                try:
                    nodemap.FindNode("ExposureTime").SetValue(exposure)
                    camera_controls["exposure"] = exposure
                except: pass
            
            if analog_gain != camera_controls["analog_gain"]:
                try:
                    nodemap.FindNode("Gain").SetValue(analog_gain)
                    camera_controls["analog_gain"] = analog_gain
                except: pass
            
            digital_gain_actual = digital_gain_x10 / 10.0
            if digital_gain_x10 != camera_controls["digital_gain"]:
                try:
                    nodemap.FindNode("DigitalGain").SetValue(digital_gain_actual)
                    camera_controls["digital_gain"] = digital_gain_x10
                except: pass
            # Draw reference circle first (behind main circle)
            if circle_state["ref_center"] is not None:
                ref_color = (255, 255, 0) if circle_state["ref_dragging"] else (255, 165, 0)  # cyan when dragging, orange otherwise
                cv2.circle(img_array, circle_state["ref_center"], circle_state["ref_radius"], ref_color, 2)
                cv2.circle(img_array, circle_state["ref_center"], 5, (255, 0, 0), -1)  # blue center point
                
                # Draw crosshair for reference
                cv2.line(img_array, (circle_state["ref_center"][0] - circle_state["ref_radius"] - 10, circle_state["ref_center"][1]),
                         (circle_state["ref_center"][0] + circle_state["ref_radius"] + 10, circle_state["ref_center"][1]), ref_color, 1)
                cv2.line(img_array, (circle_state["ref_center"][0], circle_state["ref_center"][1] - circle_state["ref_radius"] - 10),
                         (circle_state["ref_center"][0], circle_state["ref_center"][1] + circle_state["ref_radius"] + 10), ref_color, 1)
            
            # Draw main circle
            if circle_state["center"] is not None:
                color = (0, 255, 255) if circle_state["dragging"] else (0, 255, 0)  # yellow when dragging, green otherwise
                cv2.circle(img_array, circle_state["center"], circle_state["radius"], color, 2)
                cv2.circle(img_array, circle_state["center"], 3, (0, 0, 255), -1)  # red center point
                
                # Draw crosshair
                cv2.line(img_array, (circle_state["center"][0] - circle_state["radius"] - 10, circle_state["center"][1]),
                         (circle_state["center"][0] + circle_state["radius"] + 10, circle_state["center"][1]), (0, 255, 0), 1)
                cv2.line(img_array, (circle_state["center"][0], circle_state["center"][1] - circle_state["radius"] - 10),
                         (circle_state["center"][0], circle_state["center"][1] + circle_state["radius"] + 10), (0, 255, 0), 1)
                
                # Show center offset if both circles exist
                if circle_state["ref_center"] is not None:
                    # Calculate offset between centers
                    dx = circle_state["center"][0] - circle_state["ref_center"][0]
                    dy = circle_state["center"][1] - circle_state["ref_center"][1]
                    distance = np.sqrt(dx**2 + dy**2)
                    
                    # Draw line connecting centers
                    cv2.line(img_array, circle_state["center"], circle_state["ref_center"], (255, 255, 255), 1)
                    
                    # Display offset info
                    offset_text = f"Center Offset: {distance:.1f}px (X:{dx:.1f}, Y:{dy:.1f})"
                    cv2.rectangle(img_array, (10, 80), (450, 110), (0, 0, 0), -1)
                    cv2.putText(img_array, offset_text, (15, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 0), 2)
                
                # Show measurements if enabled
                if show_measure == 1:
                    radius_px = circle_state["radius"]
                    diameter_px = radius_px * 2
                    
                    # Measurement text
                    if circle_state["pixel_per_meter"] is not None:
                        radius_m = radius_px / circle_state["pixel_per_meter"]
                        diameter_m = diameter_px / circle_state["pixel_per_meter"]
                        text1 = f"Radius: {radius_px}px ({radius_m*1000:.2f}mm)"
                        text2 = f"Diameter: {diameter_px}px ({diameter_m*1000:.2f}mm)"
                    else:
                        text1 = f"Radius: {radius_px}px"
                        text2 = f"Diameter: {diameter_px}px"
                    
                    # Draw text background for readability
                    cv2.rectangle(img_array, (10, 10), (450, 70), (0, 0, 0), -1)
                    cv2.putText(img_array, text1, (15, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
                    cv2.putText(img_array, text2, (15, 55), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
            # Display instructions and status messages only if enabled
            if show_notes == 1:
                instructions = [
                    "=== CONTROLS ===",
                    "Cam: Brightness, Exposure, Gains",
                    "Calib: Circle, Measure, Specs",
                    "Mouse: L-click mark/drag | R-click clear",
                    "Keys: S=save | L=load | Q=quit",
                    "=== STATUS ==="
                ]
                
                # Calculate instruction panel size with tighter spacing
                line_spacing = 16  # reduced from 20
                instr_height = len(instructions) * line_spacing + len(status_messages) * line_spacing + 15
                cv2.rectangle(img_array, (img_width - 420, 10), (img_width - 10, instr_height), (0, 0, 0), -1)
                
                y_offset = 24
                for instr in instructions:
                    cv2.putText(img_array, instr, (img_width - 410, y_offset), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)
                    y_offset += line_spacing
                
                # Display status messages
                for msg in status_messages:
                    cv2.putText(img_array, msg[:48], (img_width - 410, y_offset),  # truncate long messages
                               cv2.FONT_HERSHEY_SIMPLEX, 0.35, (0, 255, 255), 1)
                    y_offset += line_spacing
            
            # Display current settings on image (bottom left)
            settings_y = img_height - 120
            cv2.rectangle(img_array, (10, settings_y - 5), (400, img_height - 10), (0, 0, 0), -1)
            cv2.putText(img_array, f"Brightness: {brightness}", (15, settings_y + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
            cv2.putText(img_array, f"Exposure: {exposure}", (15, settings_y + 40), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
            cv2.putText(img_array, f"Analog Gain: {analog_gain}", (15, settings_y + 60), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
            cv2.putText(img_array, f"Digital Gain: {digital_gain_actual:.1f}", (15, settings_y + 80), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
            cv2.putText(img_array, f"Circle Radius: {circle_state['radius']}px", (15, settings_y + 100), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 1)
            
            cv2.imshow(window_name, img_array)
           
            # שחרור הבאפר
            data_stream.QueueBuffer(buffer)

            # keyboard controls
            key = cv2.waitKey(1) & 0xFF
            if key == ord('q'):
                break
            elif key == ord('s'):  # save calibration
                save_calibration()
            elif key == ord('l'):  # load calibration
                load_calibration()
                cv2.setTrackbarPos("Radius", calib_window, circle_state["radius"])
                cv2.setTrackbarPos("Ref Radius", calib_window, circle_state["ref_radius"])

    except Exception as e:
        add_status_message(f"שגיאה: {e}")

    finally:
        add_status_message("סוגר משאבים...")
        if data_stream is not None:
            try:
                nodemap.FindNode("AcquisitionStop").Execute()
                data_stream.StopAcquisition(ids_peak.AcquisitionStopMode_Default)
                data_stream.Flush(ids_peak.DataStreamFlushMode_DiscardAll)
            except: pass
       
        if device is not None:
            del device
           
        cv2.destroyAllWindows()
        ids_peak.Library.Close()
        add_status_message("המערכת שוחררה.")



run_live_camera()

המצלמה עובדת!
GUI: Camera Controls & Calibration Settings
Left-click: mark/drag circle | Right-click: clear
Keyboard: S-save | L-load | Q-quit
Calibration loaded from calibration_mark.json
Reference circle at: (528, 256)
Reference circle moved to: (393, 306)
Main circle moved to: (561, 305)
Main circle moved to: (503, 407)
Reference circle moved to: (507, 402)
סוגר משאבים...
המערכת שוחררה.


KeyboardInterrupt: 